In [1]:
from algorithmV1 import Encoding

import pickle
import numpy as np
import matplotlib.pyplot as plt

from scipy.io.wavfile import read
from scipy.signal import spectrogram
import scipy.signal

fs, s = read("samples\Frisk - Au.Ra.wav")
size = 128
noverlap = 32
# nperseg si la fenêtre n'a pas directement la bonne taille
window = scipy.signal.get_window("boxcar", size, fftbins=True)

### Question 1

In [35]:
fingerprint = Encoding(window, size)
spectro, peak = fingerprint.process(fs, s)
f, t, Sxx = spectro


Sxx = Sxx.flatten()

Sxx.sort()
print(Sxx)
total_energy = float(sum(Sxx))
energy, i = 0., 1
while energy/total_energy < 0.9:
    energy += float(Sxx[-i])
    # print(Sxx[-i])
    i += 1

print(Sxx.shape)
print(i)

[     0.        0.        0.   ... 607869.1  648005.   698070.25]
(10774790,)
444724


444724 points sur 10 774 790